# Reporting Dashboard Setup for Tableau

This notebook prepares the final dataset and loads it into PostgreSQL so Tableau can:
- Access cleaned and engineered features
- Build dashboards for fraud detection analytics


In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine

# File path
project_dir = r"C:\Users\Cloud\OneDrive\Desktop\Fraud_Analytics_Project"
feature_file = os.path.join(project_dir, "data", "features", "engineered_insurance_claims.csv")

# PostgreSQL credentials
pg_user = "postgres"
pg_password = "your_password_here"
pg_host = "localhost"
pg_port = 5432
pg_dbname = "fraud_analytics"

# Connection engine
engine = create_engine(f"postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_dbname}")

## Step 1: Load Engineered Data into PostgreSQL Table

In [ ]:
df = pd.read_csv(feature_file)
df.to_sql("tableau_insurance_claims", engine, if_exists="replace", index=False)
print("✅ Data loaded into PostgreSQL table: tableau_insurance_claims")

## Step 2 (Optional): Create a PostgreSQL View for Tableau

Use pgAdmin 4 or a SQL script to create a clean view with renamed or simplified fields.

### 📜 Example SQL View (Run in pgAdmin 4)

```sql
CREATE OR REPLACE VIEW v_fraud_summary AS
SELECT
    fraud_reported,
    total_claim_amount,
    risk_score,
    fraud_weight,
    incident_year,
    incident_month,
    incident_dayofweek,
    incident_is_weekend,
    incident_type_Property_Damage,
    incident_type_Single_Vehicle_Collision,
    incident_type_Vehicle_Theft,
    collision_type_Rear_Collision,
    collision_type_Side_Collision,
    police_report_available_YES
FROM
    tableau_insurance_claims;

In [ ]:

---

### ✅ Markdown Cell 6 – Connect Tableau to PostgreSQL

```markdown
## 🔌 Step 3: Connect Tableau to PostgreSQL

1. Open Tableau Desktop
2. Select **PostgreSQL** as your data source
3. Enter connection details:
   - **Server**: `localhost`
   - **Port**: `5432`
   - **Database**: `fraud_analytics`
   - **Username**: `postgres`
   - **Password**: your_password_here
4. Choose:
   - Table: `tableau_insurance_claims`
   - OR View: `v_fraud_summary`
5. Click **"Sheet"** to start building visuals!

## Dashboard Ideas

- Total Claim Amount by Fraud Status
- Average Risk Score by Incident Type
- Fraud Trend by Month or Day of Week
- Heatmap: Fraud by Hour of Day vs Collision Type
- Bar Chart: Fraud % by Police Report Availability

### Once you're in Tableau, you can drag-and-drop fields and use filters for:

- Creating fraud dashboards
- Exporting reports
- Building presentations